In [1]:
import json, os
import tempfile
from pathlib import Path
from subprocess import Popen
from sys import stderr
from zipfile import ZipFile

import wget
import pandas as pd
import csv
import re

from openie import StanfordOpenIE
from stanfordcorenlp import StanfordCoreNLP

In [2]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

def get_sentiment3(inputtext):
    res = nlp.annotate(str(inputtext), properties={'annotators': 'sentiment', 'outputFormat': 'json', 'timeout': 10000})
    for s in res["sentences"]:
        return(s["sentimentValue"])

In [3]:
############################################

In [4]:
data_file='vaccine_segSent.csv'
    
df=pd.read_csv(data_file, header=0, encoding="utf-8", engine='python')

df_texts=df['seg_text'].tolist()
texts=df['seg_text'].tolist()

post_id=df['postInd'].tolist()
senten_id=df['sentenceInd'].tolist()

print(len(texts))

28242


In [5]:
import string, re
def preprocessing(line):
    #line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), "", line)
    return line

In [6]:
############################################################################################

In [7]:
postInd=[]
sentenceInd=[]
tripleInd=[]

sentiments=[]
subjlist=[]
verblist=[]
objelist=[]

triple_string=[]

with StanfordOpenIE() as client:
    triples=[]
    tripleCount=0
    
    i=0
    for p, sid, st in zip(post_id, senten_id, texts):
        st=str(st)
        st=re.sub("[\(\[].*?[\)\]]", "", st)
        st=re.sub(" +", " ", st)
        st=st.strip()
        
        #if len(st.split(" "))<3:
            #continue
##############################################################################################
        s1=""
        j=0
        for triple in client.annotate(str(st)):
            #if j>10:
                #break

            s1=s1+str(triple)
            tripleCount+=1
            
            postInd.append(p)
            sentenceInd.append(sid)
            tripleInd.append(j)
            
            j+=1
##############################################################################################
        s1=s1.lower()
        if len(s1)>0:
            s1=s1.strip()
            s1=s1[1:-1]
            
            s1=re.sub("president real donald trump", "trump", s1)
            s1=re.sub("president donald trump", "trump", s1)
            s1=re.sub("president trump", "trump", s1)
            s1=re.sub("real donald trump", "trump", s1)
            s1=re.sub("donald trump", "trump", s1)
            s1=re.sub("real trump", "trump", s1)
            s1=re.sub("real donald", "trump", s1)
            
            tlist=s1.split("}{")
            ############################################################################
            for s2 in tlist:
                testt=str(s2).split(", ")

                triple_str=[]
                for x in testt:
                    x=re.sub("at_time", "", x)
                    
                    #x=preprocessing(x)
                    
                    x=re.sub(" +", " ", x)
                    x=x.strip()
                    triple_str.append(x)

                #subject
                subj=triple_str[0][12:-1]
                subj=subj.strip()
                subjlist.append(subj)

                #verb
                verb=triple_str[1][13:-1]
                verb=re.sub(" +", " ", verb)
                verb=verb.strip()
                verblist.append(verb)

                verblast=verb.split(" ")[-1]

                #object
                obje=triple_str[2][11:-1]

                objefirst=obje.split(" ")[0]

                if verblast==objefirst:
                    obje=obje[len(objefirst)+1:]

                obje=re.sub(" +", " ", obje)
                obje=obje.strip()
                objelist.append(obje)
                
                sss=str(subj+" "+verb+" "+obje)
                triple_string.append(sss)
#################################################################################################
        if len(s1)>0:
            i+=1
        triples.append(s1)

print("number of triples:", tripleCount)

Starting server with command: java -Xmx8G -cp C:\Users\cl524\.stanfordnlp_resources\stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-eeaf4a2e002248a3.props -preload openie
number of triples: 101439


In [8]:
len(subjlist)==len(verblist)==len(objelist)

True

In [9]:
print(len(subjlist))

101439


In [10]:
############################################################################

In [11]:
tripleoutputs = list(zip(postInd, sentenceInd, tripleInd, subjlist, verblist, objelist))

from pandas import DataFrame

df_tripleoutputs = DataFrame(tripleoutputs, columns=['postInd', 'sentenceInd', 'tripleInd', 'subject', 'relation', 'object'])

#df_tripleoutputs.to_csv ("vaccine_segSent_triples.csv", index = None, header=True)

In [12]:
'''
from pandas import DataFrame

c={'text': pd.Series(texts), 'triples': pd.Series(triples)}
df_results = pd.DataFrame(c)
df_results.to_csv ("vaccine_segSent_triples_2.csv", index = None, header=True)
'''

'\nfrom pandas import DataFrame\n\nc={\'text\': pd.Series(texts), \'triples\': pd.Series(triples)}\ndf_results = pd.DataFrame(c)\ndf_results.to_csv ("vaccine_segSent_triples_2.csv", index = None, header=True)\n'

In [13]:
####################################################################################################################
####################################################################################################################
####################################################################################################################
####################################################################################################################
####################################################################################################################

In [14]:
df=df_tripleoutputs

In [15]:
#data_file='vaccine_segSent_triples.csv'
    
#df=pd.read_csv(data_file, header=0, encoding="utf-8", engine='python')

In [16]:
df["triple_string"] = df["subject"]+" " +df["relation"]+" "+df['object']

In [17]:
df['sentiment']=df["triple_string"].apply(get_sentiment3)

In [18]:
df.to_csv ("vaccine_segSent_triples_senti.csv", index = None, header=True)